In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {
               'price':'{:.2f}','buy_p':'{:.2f}','sell_p':'{:.2f}','diff':'{:.2f}',
               'ppu':'{:.4f}',
               'net':'{:,.2f}','fee':'{:,.2f}','vat':'{:,.2f}',
               'buy_amt':'{:,.2f}','sell_amt':'{:,.2f}','profit':'{:,.2f}',
               'div_amt':'{:,.2f}','ttl_amt':'{:,.2f}','pct':'{:,.2f}',
               'qty':'{:,}','ttl_qty':'{:,}','number':'{:,}','days':'{:,}'
              }
pd.set_option("display.max_rows", None)

cols = 'name ttl_qty buy_amt ppu div_amt buy_date x_date p_date days pct year quarter'.split()
colt = 'name ttl_qty buy_amt ppu div_amt buy_date x_date p_date days pct year quarter number'.split()

name = 'JASIF'

### 1ST Batch

In [2]:
div_year = 2018
div_quarter = 1
ids = '515'

In [3]:
sql = '''
SELECT T.name, SUM(qty) AS ttl_qty, 
SUM(B.net) AS buy_amt, MIN(date) AS buy_date
FROM buys B
JOIN stocks T ON B.stock_id = T.id 
WHERE B.id IN (%s)
GROUP BY T.name'''
sql = sql % (ids)
buys = pd.read_sql(sql, conpf)
buys['ttl_qty'] = buys.ttl_qty.astype(int)
buys.style.format(format_dict)

,name,ttl_qty,buy_amt,buy_date
0,JASIF,"50,000","501,107.45",2018-05-17


In [4]:
sql = """
SELECT id, name, year, quarter, number, ppu, net AS div_amt, x_date, p_date
FROM dividends
WHERE name = '%s'
AND ((year = %s AND quarter >= %s) 
OR (year > %s))
ORDER BY name, year DESC, quarter DESC
"""
sql = sql % (name, div_year, div_quarter, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,241,JASIF,2021,1,"130,000",0.2400,"31,200.00",2021-05-21,2021-06-05
1,198,JASIF,2020,4,"130,000",0.2500,"32,500.00",2021-02-24,2021-03-15
2,191,JASIF,2020,3,"130,000",0.2500,"32,500.00",2020-11-18,2020-12-04
3,101,JASIF,2020,2,"100,000",0.2500,"25,000.00",2020-09-02,2020-09-22
4,5,JASIF,2020,1,"100,000",0.2400,"24,000.00",2020-05-27,2020-06-15
5,30,JASIF,2019,4,"100,000",0.2000,"20,000.00",2020-02-17,2020-03-04
6,37,JASIF,2019,3,"90,000",0.2300,"20,700.00",2019-11-14,2019-12-02
7,50,JASIF,2019,2,"60,000",0.2300,"13,800.00",2019-08-19,2019-09-04
8,53,JASIF,2019,1,"60,000",0.2400,"14,400.00",2019-05-22,2019-06-07
9,75,JASIF,2018,4,"60,000",0.2300,"13,800.00",2019-03-01,2019-03-19


In [5]:
df_merge = pd.merge(buys, dividends, on='name', how='inner')
df_merge['div_amt'] = df_merge.ppu * df_merge.ttl_qty
df_merge['days'] = df_merge.p_date - df_merge.buy_date
df_merge['days'] = df_merge['days']/np.timedelta64(1,'D')
df_merge['days'] = df_merge['days'].astype(int)
df_merge['pct'] = round(df_merge['div_amt']/df_merge['buy_amt']*36500/df_merge['days'],2)
df_merge[cols]

,name,ttl_qty,buy_amt,ppu,div_amt,buy_date,x_date,p_date,days,pct,year,quarter
0,JASIF,50000,501107.45,0.24,12000.0,2018-05-17,2021-05-21,2021-06-05,1115,0.78,2021,1
1,JASIF,50000,501107.45,0.25,12500.0,2018-05-17,2021-02-24,2021-03-15,1033,0.88,2020,4
2,JASIF,50000,501107.45,0.25,12500.0,2018-05-17,2020-11-18,2020-12-04,932,0.98,2020,3
3,JASIF,50000,501107.45,0.25,12500.0,2018-05-17,2020-09-02,2020-09-22,859,1.06,2020,2
4,JASIF,50000,501107.45,0.24,12000.0,2018-05-17,2020-05-27,2020-06-15,760,1.15,2020,1
5,JASIF,50000,501107.45,0.20,10000.0,2018-05-17,2020-02-17,2020-03-04,657,1.11,2019,4
6,JASIF,50000,501107.45,0.23,11500.0,2018-05-17,2019-11-14,2019-12-02,564,1.49,2019,3
7,JASIF,50000,501107.45,0.23,11500.0,2018-05-17,2019-08-19,2019-09-04,475,1.76,2019,2
8,JASIF,50000,501107.45,0.24,12000.0,2018-05-17,2019-05-22,2019-06-07,386,2.26,2019,1
9,JASIF,50000,501107.45,0.23,11500.0,2018-05-17,2019-03-01,2019-03-19,306,2.74,2018,4


In [6]:
df_grp =df_merge.groupby(['name'])['div_amt'].sum()
div1 = df_grp
div1

name
JASIF    151500.0
Name: div_amt, dtype: float64

In [7]:
file_name = 'div-' + name + '.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

('../data/div-JASIF.csv',
 '\\Users\\User\\iCloudDrive\\div-JASIF.csv',
 '\\Users\\User\\Dropbox\\div-JASIF.csv')

In [8]:
df_merge[colt].to_csv(output_file)
df_merge[colt].to_csv(data_file)
df_merge[colt].to_csv(box_file)

### Second batch of purchase

In [9]:
div_year = 2018
div_quarter = 2
ids = '516'

In [10]:
sql = '''
SELECT T.name, SUM(qty) AS ttl_qty, 
SUM(B.net) AS buy_amt, MIN(date) AS buy_date
FROM buys B
JOIN stocks T ON B.stock_id = T.id 
WHERE B.id IN (%s)
GROUP BY T.name'''
sql = sql % (ids)
buys = pd.read_sql(sql, conpf)
buys['ttl_qty'] = buys.ttl_qty.astype(int)
buys.style.format(format_dict)

,name,ttl_qty,buy_amt,buy_date
0,JASIF,"10,000","100,221.49",2018-08-01


In [11]:
sql = """
SELECT id, name, year, quarter, number, ppu, net AS div_amt, x_date, p_date
FROM dividends
WHERE name = '%s'
AND ((year = %s AND quarter >= %s) 
OR (year > %s))
ORDER BY name, year DESC, quarter DESC
"""
sql = sql % (name, div_year, div_quarter, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,241,JASIF,2021,1,"130,000",0.2400,"31,200.00",2021-05-21,2021-06-05
1,198,JASIF,2020,4,"130,000",0.2500,"32,500.00",2021-02-24,2021-03-15
2,191,JASIF,2020,3,"130,000",0.2500,"32,500.00",2020-11-18,2020-12-04
3,101,JASIF,2020,2,"100,000",0.2500,"25,000.00",2020-09-02,2020-09-22
4,5,JASIF,2020,1,"100,000",0.2400,"24,000.00",2020-05-27,2020-06-15
5,30,JASIF,2019,4,"100,000",0.2000,"20,000.00",2020-02-17,2020-03-04
6,37,JASIF,2019,3,"90,000",0.2300,"20,700.00",2019-11-14,2019-12-02
7,50,JASIF,2019,2,"60,000",0.2300,"13,800.00",2019-08-19,2019-09-04
8,53,JASIF,2019,1,"60,000",0.2400,"14,400.00",2019-05-22,2019-06-07
9,75,JASIF,2018,4,"60,000",0.2300,"13,800.00",2019-03-01,2019-03-19


In [12]:
df_merge = pd.merge(buys, dividends, on='name', how='inner')
df_merge['div_amt'] = df_merge.ppu * df_merge.ttl_qty
df_merge['days'] = df_merge.p_date - df_merge.buy_date
df_merge['days'] = df_merge['days']/np.timedelta64(1,'D')
df_merge['days'] = df_merge['days'].astype(int)
df_merge['pct'] = round(df_merge['div_amt']/df_merge['buy_amt']*36500/df_merge['days'],2)
df_merge[cols]

,name,ttl_qty,buy_amt,ppu,div_amt,buy_date,x_date,p_date,days,pct,year,quarter
0,JASIF,10000,100221.49,0.24,2400.0,2018-08-01,2021-05-21,2021-06-05,1039,0.84,2021,1
1,JASIF,10000,100221.49,0.25,2500.0,2018-08-01,2021-02-24,2021-03-15,957,0.95,2020,4
2,JASIF,10000,100221.49,0.25,2500.0,2018-08-01,2020-11-18,2020-12-04,856,1.06,2020,3
3,JASIF,10000,100221.49,0.25,2500.0,2018-08-01,2020-09-02,2020-09-22,783,1.16,2020,2
4,JASIF,10000,100221.49,0.24,2400.0,2018-08-01,2020-05-27,2020-06-15,684,1.28,2020,1
5,JASIF,10000,100221.49,0.20,2000.0,2018-08-01,2020-02-17,2020-03-04,581,1.25,2019,4
6,JASIF,10000,100221.49,0.23,2300.0,2018-08-01,2019-11-14,2019-12-02,488,1.72,2019,3
7,JASIF,10000,100221.49,0.23,2300.0,2018-08-01,2019-08-19,2019-09-04,399,2.10,2019,2
8,JASIF,10000,100221.49,0.24,2400.0,2018-08-01,2019-05-22,2019-06-07,310,2.82,2019,1
9,JASIF,10000,100221.49,0.23,2300.0,2018-08-01,2019-03-01,2019-03-19,230,3.64,2018,4


In [13]:
df_grp =df_merge.groupby(['name'])['div_amt'].sum()
div2 = df_grp
div2

name
JASIF    28100.0
Name: div_amt, dtype: float64

In [14]:
df_merge[colt].to_csv(output_file,mode='a',header=False)
df_merge[colt].to_csv(data_file,mode='a',header=False)
df_merge[colt].to_csv(box_file,mode='a',header=False)

### Third batch of purchase (must change qty of record 516 to zero)

In [15]:
div_year = 2019
div_quarter = 3
ids = '517'

In [16]:
sql = '''
SELECT T.name, SUM(qty) AS ttl_qty, 
SUM(B.net) AS buy_amt, MIN(date) AS buy_date
FROM buys B
JOIN stocks T ON B.stock_id = T.id 
WHERE B.id IN (%s)
GROUP BY T.name'''
sql = sql % (ids)
buys = pd.read_sql(sql, conpf)
buys['ttl_qty'] = buys.ttl_qty.astype(int)
buys.style.format(format_dict)

,name,ttl_qty,buy_amt,buy_date
0,JASIF,"30,000","300,664.47",2019-11-01


In [17]:
sql = """
SELECT id, name, year, quarter, number, ppu, net AS div_amt, x_date, p_date
FROM dividends
WHERE name = '%s'
AND ((year = %s AND quarter >= %s) 
OR (year > %s))
ORDER BY name, year DESC, quarter DESC
"""
sql = sql % (name, div_year, div_quarter, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,241,JASIF,2021,1,"130,000",0.2400,"31,200.00",2021-05-21,2021-06-05
1,198,JASIF,2020,4,"130,000",0.2500,"32,500.00",2021-02-24,2021-03-15
2,191,JASIF,2020,3,"130,000",0.2500,"32,500.00",2020-11-18,2020-12-04
3,101,JASIF,2020,2,"100,000",0.2500,"25,000.00",2020-09-02,2020-09-22
4,5,JASIF,2020,1,"100,000",0.2400,"24,000.00",2020-05-27,2020-06-15
5,30,JASIF,2019,4,"100,000",0.2000,"20,000.00",2020-02-17,2020-03-04
6,37,JASIF,2019,3,"90,000",0.2300,"20,700.00",2019-11-14,2019-12-02


In [18]:
df_merge = pd.merge(buys, dividends, on='name', how='inner')
df_merge['div_amt'] = df_merge.ppu * df_merge.ttl_qty
df_merge['days'] = df_merge.p_date - df_merge.buy_date
df_merge['days'] = df_merge['days']/np.timedelta64(1,'D')
df_merge['days'] = df_merge['days'].astype(int)
df_merge['pct'] = round(df_merge['div_amt']/df_merge['buy_amt']*36500/df_merge['days'],2)
df_merge[cols]

,name,ttl_qty,buy_amt,ppu,div_amt,buy_date,x_date,p_date,days,pct,year,quarter
0,JASIF,30000,300664.47,0.24,7200.0,2019-11-01,2021-05-21,2021-06-05,582,1.50,2021,1
1,JASIF,30000,300664.47,0.25,7500.0,2019-11-01,2021-02-24,2021-03-15,500,1.82,2020,4
2,JASIF,30000,300664.47,0.25,7500.0,2019-11-01,2020-11-18,2020-12-04,399,2.28,2020,3
3,JASIF,30000,300664.47,0.25,7500.0,2019-11-01,2020-09-02,2020-09-22,326,2.79,2020,2
4,JASIF,30000,300664.47,0.24,7200.0,2019-11-01,2020-05-27,2020-06-15,227,3.85,2020,1
5,JASIF,30000,300664.47,0.20,6000.0,2019-11-01,2020-02-17,2020-03-04,124,5.87,2019,4
6,JASIF,30000,300664.47,0.23,6900.0,2019-11-01,2019-11-14,2019-12-02,31,27.02,2019,3


In [19]:
df_grp =df_merge.groupby(['name'])['div_amt'].sum()
div3 = df_grp
div3

name
JASIF    49800.0
Name: div_amt, dtype: float64

In [20]:
df_merge[colt].to_csv(output_file,mode='a',header=False)
df_merge[colt].to_csv(data_file,mode='a',header=False)
df_merge[colt].to_csv(box_file,mode='a',header=False)

### 4th Batch

In [21]:
div_year = 2019
div_quarter = 4
ids = '767'

In [22]:
sql = '''
SELECT T.name, SUM(qty) AS ttl_qty, 
SUM(B.net) AS buy_amt, MIN(date) AS buy_date
FROM buys B
JOIN stocks T ON B.stock_id = T.id 
WHERE B.id IN (%s)
GROUP BY T.name'''
sql = sql % (ids)
buys = pd.read_sql(sql, conpf)
buys['ttl_qty'] = buys.ttl_qty.astype(int)
buys.style.format(format_dict)

,name,ttl_qty,buy_amt,buy_date
0,JASIF,"10,000","100,221.49",2020-02-01


In [23]:
sql = """
SELECT id, name, year, quarter, number, ppu, net AS div_amt, x_date, p_date
FROM dividends
WHERE name = '%s'
AND ((year = %s AND quarter >= %s) 
OR (year > %s))
ORDER BY name, year DESC, quarter DESC
"""
sql = sql % (name, div_year, div_quarter, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,241,JASIF,2021,1,"130,000",0.2400,"31,200.00",2021-05-21,2021-06-05
1,198,JASIF,2020,4,"130,000",0.2500,"32,500.00",2021-02-24,2021-03-15
2,191,JASIF,2020,3,"130,000",0.2500,"32,500.00",2020-11-18,2020-12-04
3,101,JASIF,2020,2,"100,000",0.2500,"25,000.00",2020-09-02,2020-09-22
4,5,JASIF,2020,1,"100,000",0.2400,"24,000.00",2020-05-27,2020-06-15
5,30,JASIF,2019,4,"100,000",0.2000,"20,000.00",2020-02-17,2020-03-04


In [24]:
df_merge = pd.merge(buys, dividends, on='name', how='inner')
df_merge['div_amt'] = df_merge.ppu * df_merge.ttl_qty
df_merge['days'] = df_merge.p_date - df_merge.buy_date
df_merge['days'] = df_merge['days']/np.timedelta64(1,'D')
df_merge['days'] = df_merge['days'].astype(int)
df_merge['pct'] = round(df_merge['div_amt']/df_merge['buy_amt']*36500/df_merge['days'],2)
df_merge[cols]

,name,ttl_qty,buy_amt,ppu,div_amt,buy_date,x_date,p_date,days,pct,year,quarter
0,JASIF,10000,100221.49,0.24,2400.0,2020-02-01,2021-05-21,2021-06-05,490,1.78,2021,1
1,JASIF,10000,100221.49,0.25,2500.0,2020-02-01,2021-02-24,2021-03-15,408,2.23,2020,4
2,JASIF,10000,100221.49,0.25,2500.0,2020-02-01,2020-11-18,2020-12-04,307,2.97,2020,3
3,JASIF,10000,100221.49,0.25,2500.0,2020-02-01,2020-09-02,2020-09-22,234,3.89,2020,2
4,JASIF,10000,100221.49,0.24,2400.0,2020-02-01,2020-05-27,2020-06-15,135,6.47,2020,1
5,JASIF,10000,100221.49,0.20,2000.0,2020-02-01,2020-02-17,2020-03-04,32,22.76,2019,4


In [25]:
df_grp =df_merge.groupby(['name'])['div_amt'].sum()
div4 = df_grp
div4

name
JASIF    14300.0
Name: div_amt, dtype: float64

In [26]:
df_merge[colt].to_csv(output_file,mode='a',header=False)
df_merge[colt].to_csv(data_file,mode='a',header=False)
df_merge[colt].to_csv(box_file,mode='a',header=False)

### 5th Batch

In [27]:
div_year = 2020
div_quarter = 3
ids = '769'

In [28]:
sql = '''
SELECT T.name, SUM(qty) AS ttl_qty, 
SUM(B.net) AS buy_amt, MIN(date) AS buy_date
FROM buys B
JOIN stocks T ON B.stock_id = T.id 
WHERE B.id IN (%s)
GROUP BY T.name'''
sql = sql % (ids)
buys = pd.read_sql(sql, conpf)
buys['ttl_qty'] = buys.ttl_qty.astype(int)
buys.style.format(format_dict)

,name,ttl_qty,buy_amt,buy_date
0,JASIF,"30,000","300,664.47",2020-11-01


In [29]:
sql = """
SELECT id, name, year, quarter, number, ppu, net AS div_amt, x_date, p_date
FROM dividends
WHERE name = '%s'
AND ((year = %s AND quarter >= %s) 
OR (year > %s))
ORDER BY name, year DESC, quarter DESC
"""
sql = sql % (name, div_year, div_quarter, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,241,JASIF,2021,1,"130,000",0.2400,"31,200.00",2021-05-21,2021-06-05
1,198,JASIF,2020,4,"130,000",0.2500,"32,500.00",2021-02-24,2021-03-15
2,191,JASIF,2020,3,"130,000",0.2500,"32,500.00",2020-11-18,2020-12-04


In [30]:
df_merge = pd.merge(buys, dividends, on='name', how='inner')
df_merge['div_amt'] = df_merge.ppu * df_merge.ttl_qty
df_merge['days'] = df_merge.p_date - df_merge.buy_date
df_merge['days'] = df_merge['days']/np.timedelta64(1,'D')
df_merge['days'] = df_merge['days'].astype(int)
df_merge['pct'] = round(df_merge['div_amt']/df_merge['buy_amt']*36500/df_merge['days'],2)
df_merge[cols]

,name,ttl_qty,buy_amt,ppu,div_amt,buy_date,x_date,p_date,days,pct,year,quarter
0,JASIF,30000,300664.47,0.24,7200.0,2020-11-01,2021-05-21,2021-06-05,216,4.05,2021,1
1,JASIF,30000,300664.47,0.25,7500.0,2020-11-01,2021-02-24,2021-03-15,134,6.79,2020,4
2,JASIF,30000,300664.47,0.25,7500.0,2020-11-01,2020-11-18,2020-12-04,33,27.59,2020,3


In [31]:
df_grp =df_merge.groupby(['name'])['div_amt'].sum()
div5 = df_grp
div5

name
JASIF    22200.0
Name: div_amt, dtype: float64

In [32]:
df_merge[colt].to_csv(output_file,mode='a',header=False)
df_merge[colt].to_csv(data_file,mode='a',header=False)
df_merge[colt].to_csv(box_file,mode='a',header=False)

### End of process

In [33]:
div_year = 2018
div_quarter = 1

In [34]:
sql = """
SELECT id, name, year, quarter, number, ppu, number * ppu AS div_amt, x_date, p_date
FROM dividends
WHERE name = '%s'
AND ((year = %s AND quarter >= %s) 
OR (year > %s))
ORDER BY name, year, quarter
"""
sql = sql % (name, div_year, div_quarter, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,99,JASIF,2018,1,"60,000",0.2200,"13,200.00",2018-05-24,2018-06-12
1,93,JASIF,2018,2,"70,000",0.2300,"16,100.00",2018-08-24,2018-09-11
2,82,JASIF,2018,3,"70,000",0.2200,"15,400.00",2018-11-18,2018-12-11
3,75,JASIF,2018,4,"60,000",0.2300,"13,800.00",2019-03-01,2019-03-19
4,53,JASIF,2019,1,"60,000",0.2400,"14,400.00",2019-05-22,2019-06-07
5,50,JASIF,2019,2,"60,000",0.2300,"13,800.00",2019-08-19,2019-09-04
6,37,JASIF,2019,3,"90,000",0.2300,"20,700.00",2019-11-14,2019-12-02
7,30,JASIF,2019,4,"100,000",0.2000,"20,000.00",2020-02-17,2020-03-04
8,5,JASIF,2020,1,"100,000",0.2400,"24,000.00",2020-05-27,2020-06-15
9,101,JASIF,2020,2,"100,000",0.2500,"25,000.00",2020-09-02,2020-09-22


In [35]:
dividends.div_amt.sum()

272600.0